In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Assuming the data is in a JSON file named 'patient_data.json'
df = pd.read_json('/content/drive/MyDrive/ML_Dataset/DataEngineeringQ2.json')


In [ ]:
# Calculate missing value percentages
missing_pct = df.isnull().sum() / len(df) * 100
print(missing_pct)


_id                 0.0
appointmentId       0.0
patientDetails      0.0
phoneNumber         0.0
consultationData    0.0
dtype: float64


In [ ]:
"""1. Share percentage of missing values, for the columns firstName, lastName and DOB from
patientDetails.
Missing values are defined as blank and empty strings."""
firstNames = (df['patientDetails'].apply(lambda x: x.get('firstName')).isnull().sum() / len(df)) * 100
lastNames = (df['patientDetails'].apply(lambda x: x.get('lastName')).isnull().sum() / len(df)) * 100
birthDate = (df['patientDetails'].apply(lambda x: x.get('birthDate')).isnull().sum() / len(df)) * 100

print("Missing val for firstName:", firstNames)
print("Missing val for lastName:", lastNames)
print("Missing val for birthDate:", round(birthDate,2))

Missing val for firstName: 0.0
Missing val for lastName: 0.0
Missing val for birthDate: 32.26


In [ ]:
"""2. What is the percentage of female gender after imputation of gender column basis mode."""
df['patientDetails'] = df['patientDetails'].apply(lambda x: x if x.get('gender') else {**x, 'gender': 'Female'})

female_pct = (df['patientDetails'].apply(lambda x: x.get('gender')) == 'Female').sum() / len(df) * 100

print("Percentage of female gender:", round(female_pct,2))

Percentage of female gender: 32.26


In [ ]:
"""3. Add a column ageGroup that categorizes patients into age groups (e.g., Child for ages 0-12, Teen for
ages 13-19, Adult for ages 20-59, Senior for ages 60+). What is count of Adult?"""

!pip install pandas
import pandas as pd
df['patientDetails.age'] = pd.to_datetime(df['patientDetails'].apply(lambda x: x.get('birthDate')), errors='coerce')

def cal_age(birthdate):
    # Calculate age only if birthdate is valid
    if pd.notnull(birthdate):
        age = pd.to_datetime('today').year - birthdate.year
        if age <= 12:
            return 'Child'
        elif age <= 19:
            return 'Teen'
        elif age <= 59:
            return 'Adult'
        else:
            return 'Senior'
    else:
        return 'Unknown' # Or handle missing age group as needed

df['ageGroup'] = df['patientDetails.age'].apply(cal_age)

# Count adults
adult_count = df[df['ageGroup'] == 'Adult'].shape[0]
print("Count of adults:", adult_count)

Count of adults: 21


In [ ]:
"""4. Calculate the average number of medicines prescribed in the dataset (up to 2 decimal points)."""
df['medicine_count'] = df['consultationData'].apply(lambda x: len(set(medicine['medicineName'] for medicine in x['medicines'])))
avg_medicines_prescribed = df['medicine_count'].mean()
print("number of medicines prescribed:", round(avg_medicines_prescribed, 2))

number of medicines prescribed: 2.03


In [ ]:
"""5. Determine the 3rd most frequently prescribed medicineName"""
#print ony 3rd most frequent prescribed medicine

print(df['consultationData'].apply(lambda x: [medicine['medicineName'] for medicine in x['medicines']]).explode().value_counts().nlargest(3))

consultationData
A    17
D    17
C    13
Name: count, dtype: int64


In [ ]:
"""5. Determine the 3

rd most frequently prescribed medicineName"""
medicine_counts = df['consultationData'].apply(lambda x: [medicine['medicineName'] for medicine in x.get('medicines', [])]) \
                                       .explode().value_counts()
if len(medicine_counts) >= 3:
    # Get the third most frequent medicine
    third_most_prescribed = medicine_counts.index[2]
    print("Top 3 rd medicine:", third_most_prescribed)
else:
    print("Thank you")

Top 3 rd medicine: C


In [ ]:
"""6. Find the percentage distribution of total active and inactive medicines."""
medicine_active_status = df['consultationData'].apply(lambda x: [medicine['isActive'] for medicine in x.get('medicines', [])]).explode()

active = (medicine_active_status == True).sum() / len(medicine_active_status) * 100
inactive = (medicine_active_status == False).sum() / len(medicine_active_status) * 100
print("Percentage of active medicines:", round(active,2))
print("Percentage of inactive medicines:",round(inactive, 2) )

Percentage of active medicines: 63.89
Percentage of inactive medicines: 27.78


**LEVEL-2**

1. Add a column isValidMobile of boolean which contains the values true or false against each value from
phoneNumber column If a phone number is a valid Indian phone number.


• true : indicates number is valid
• false : indicates number is not valid

In [ ]:
#write a function to find valid number can have '+91' or '91' as prefix, a valid number must lie be between 6000000000-9999999999
#and must have 10 digits
#if valid return true else false
def validation(num):
  if num.startswith('+91') or num.startswith('91'):
    if(len(num)==10 and 6000000000<=int(num)<=9999999999):
      return True
  return False

df['isValidMobile'] = df['phoneNumber'].apply(validation)

count = df[df['isValidMobile'] == True].shape[0]
print("valid phone numbers:", count)



valid phone numbers: 1


In [ ]:
"""2. What is the pearson correlation between the number of prescribed medicines and the patient's age?"""
df['avg_medicines'] = df.explode('consultationData').apply(lambda row: len(row['consultationData'].get('medicines', [])) if isinstance(row['consultationData'], dict) else 0, axis=1).groupby(df['patientDetails.age']).mean()

# Calculate the Pearson correlation
correlation = df[['avg_medicines', 'patientDetails.age']].corr()['avg_medicines']['patientDetails.age']

print("the number of prescribed medicines and the patient's age", correlation)

the number of prescribed medicines and the patient's age nan
